In [1]:
import pandas as pd
import yfinance as yf
import datetime as dt
from pandas_datareader import data as pdr
from tkinter import Tk
import os
import numpy as np
import schedule
import time
import smtplib
from datetime import date
import yagmail
import keyring


yf.pdr_override()

In [2]:
# List of tickers to iterate through. 

dow_list = ["AXP", "AMGN", 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', "GS", "HD", "HON", "IBM", "INTC", "JNJ", "KO", "JPM", "MCD",
           "MMM", "MRK", "MSFT", "NKE", "PG", "TRV", "UNH", "CRM", "VZ", "V", "WBA", "WMT", "DIS", "DOW"]


ev_list = ["AYRO", "NRG", "IDRV", "DRIV", "JCI", "RIDE", "SOLO", "LI", "VWAGY", "NKLA", "NIO", "TSLA", "TM", 'WKHS']


semiconductor_list = ["TSM", "ASML", "QCOM", "AMD", "NXPI", "SWKS", "TXN", "MRAAY", "NXPI", "IFX", "MU", "IFNNY"] 


biotech_list = ["BDSI", "ARDX", "APTO", "AGLE", "BCRX", "ASMB", "DVAX", "WVE", "KDMN", "ISEE", "SRPT", "ALNY", "INCY",
                "ILMN", "REGN", "BMRN", "AMGN", "VRTX", "ALXN", "BIIB", "GILD", "HZNP"]

gaming_list = ["PENN", "DKNG", "CZR", "BETZ", "MGM"]


flight_list = ["UAL", "DAL", "AAL", "SAVE", "LUV", "JBLU", "ALK"]


fintech_list = ["SQ", "PYPL", "GS", "GDOT", "MELI"]


blockchain_list = ["NVDA", "CME", "SQ", "IBM", "MA", "DOCU", "AMZN"]

# Random stocks to test
random_list = ["FUNC", "SPPI", "FXO", "MGPI", "GWRS", "NUV", "FLGE", "NCMI", "ODP", "CHKEL", "OZK", "FRHC"]

energy_list = ["EOG", "LNG", "BP", "EPD", "SU", "CVX", "RUN", "ENPH", "DVN"]

infrastructure_list = ['FAST', "MLM", "NSC", "UNP"]

total_list = (dow_list + ev_list + semiconductor_list + biotech_list + flight_list + fintech_list + blockchain_list +
             energy_list + infrastructure_list)

In [3]:
# Creates an empty dictionary, then grabs the data from yahoo finance

start_date = dt.datetime.now() - dt.timedelta(days=90)
end_date = dt.datetime.now()
ticker_dictionary = {}

# Specify chosen list of stocks below
for i in total_list:
    tickerSymbol = i
    tickerData = yf.Ticker(tickerSymbol)
    tickerDF = tickerData.history(period = '1d', start=start_date, end=end_date)
    ticker_dictionary[i] = tickerDF

- IFX: No data found for this date range, symbol may be delisted


In [4]:
percent_change = []

def buy_sell(signal):
    Buy = []
    Sell = []
    purchase = 0
    for i in range(0, len(signal)):
        Close = df["Close"][i]
        Close = round(Close, 2)
        
        if signal['MACD'][i] > signal['Signal Line'][i]:
            Sell.append(np.nan)
            
            if purchase != 1:
                Buy.append(signal['Close'][i])
                purchase = 1
                bp = Close
                print("Buying @ " + str(round(Close, 2)) + " on " + str(df.index[i]))

            else:
                Buy.append(np.nan)

        elif signal['MACD'][i] < signal['Signal Line'][i]:
            Buy.append(np.nan)
            
            if purchase != 0:
                Sell.append(signal['Close'][i])
                purchase = 0
                sp = Close
                pc = (sp/bp - 1) * 100
                pc = round(pc, 2)
                percent_change.append(pc)
                print("Sell now @ " +str(round(Close, 2)) + " on " + str(df.index[i]))
                
            else:
                Sell.append(np.nan)

        else:
            Sell.append(np.nan)
            Buy.append(np.nan)
         

            
    
    print(percent_change)
    print("     TOTAL GAINS/LOSSES: " + str(sum(percent_change)) + "%")

    
    return(Buy, Sell)

In [5]:
total_change = 0

# calculate short/long term EMA, MACD, and signal line indicators

for i in ticker_dictionary:

    df = ticker_dictionary[i]

# the short and long EMAs can easily be changed by changing span to other numbers.

# short ema 

    shortEMA = df.Close.ewm(span=8, adjust=False).mean()

# long ema

    longEMA = df.Close.ewm(span=21, adjust=False).mean()

# MACD line

    MACD = shortEMA - longEMA

# signal line

    signal_line = MACD.ewm(span=5, adjust = False).mean()

# Create new columns for data
    df["MACD"] = MACD
    df['Signal Line'] = signal_line

    percent_change = []
    
    
    print("****CHECKING STOCK " + str(i) + "****")
    
# call buy_sell function
    a = buy_sell(df)
    
# Calculate average gains/losses
    #total_change =  sum(percent_change) / len(percent_change)
    df['Buy_Signal_Price'] = a[0]
    df['Sell_Signal_Price'] = a[1]
    
print("")
print("Total Group Stock Gains/Losses: " + str(round(total_change, 2)) + "%")

****CHECKING STOCK AXP****
Buying @ 121.9 on 2020-12-02 00:00:00
Sell now @ 121.41 on 2020-12-10 00:00:00
Buying @ 117.95 on 2020-12-28 00:00:00
Sell now @ 121.15 on 2021-01-25 00:00:00
Buying @ 124.9 on 2021-02-04 00:00:00
[-0.4, 2.71]
     TOTAL GAINS/LOSSES: 2.31%
****CHECKING STOCK AMGN****
Buying @ 224.89 on 2020-12-02 00:00:00
Sell now @ 226.8 on 2020-12-18 00:00:00
Buying @ 225.49 on 2020-12-30 00:00:00
Sell now @ 245.92 on 2021-01-28 00:00:00
[0.85, 9.06]
     TOTAL GAINS/LOSSES: 9.91%
****CHECKING STOCK AAPL****
Buying @ 122.9 on 2020-12-02 00:00:00
Sell now @ 122.07 on 2020-12-04 00:00:00
Buying @ 123.57 on 2020-12-07 00:00:00
Sell now @ 121.6 on 2020-12-09 00:00:00
Buying @ 127.69 on 2020-12-15 00:00:00
Sell now @ 129.22 on 2021-01-04 00:00:00
Buying @ 136.67 on 2021-01-21 00:00:00
Sell now @ 131.76 on 2021-01-29 00:00:00
[-0.68, -1.59, 1.2, -3.59]
     TOTAL GAINS/LOSSES: -4.66%
****CHECKING STOCK BA****
Buying @ 223.85 on 2020-12-02 00:00:00
Sell now @ 228.62 on 2020-12-14

****CHECKING STOCK CRM****
Buying @ 223.62 on 2020-12-16 00:00:00
Sell now @ 222.4 on 2020-12-30 00:00:00
Buying @ 222.04 on 2021-01-08 00:00:00
Sell now @ 218.25 on 2021-01-11 00:00:00
Buying @ 223.44 on 2021-01-20 00:00:00
Sell now @ 240.95 on 2021-02-22 00:00:00
[-0.55, -1.71, 7.84]
     TOTAL GAINS/LOSSES: 5.58%
****CHECKING STOCK VZ****
Buying @ 60.71 on 2020-12-02 00:00:00
Sell now @ 59.87 on 2020-12-10 00:00:00
Buying @ 57.96 on 2021-01-05 00:00:00
Sell now @ 57.46 on 2021-01-11 00:00:00
Buying @ 57.26 on 2021-01-20 00:00:00
Sell now @ 55.13 on 2021-01-27 00:00:00
Buying @ 55.32 on 2021-02-05 00:00:00
Sell now @ 55.3 on 2021-02-26 00:00:00
[-1.38, -0.86, -3.72, -0.04]
     TOTAL GAINS/LOSSES: -6.0%
****CHECKING STOCK V****
Buying @ 212.35 on 2020-12-04 00:00:00
Sell now @ 209.26 on 2020-12-09 00:00:00
Buying @ 210.85 on 2020-12-17 00:00:00
Sell now @ 205.52 on 2020-12-22 00:00:00
Buying @ 212.3 on 2020-12-28 00:00:00
Sell now @ 213.48 on 2021-01-07 00:00:00
Buying @ 202.3 on 202

Sell now @ 157.47 on 2021-01-13 00:00:00
Buying @ 162.51 on 2021-01-20 00:00:00
Sell now @ 155.44 on 2021-01-27 00:00:00
Buying @ 168.8 on 2021-01-29 00:00:00
Sell now @ 187.91 on 2021-02-18 00:00:00
[-7.28, 6.54, -4.35, 11.32]
     TOTAL GAINS/LOSSES: 6.23%
****CHECKING STOCK TXN****
Buying @ 165.52 on 2020-12-04 00:00:00
Sell now @ 161.02 on 2020-12-10 00:00:00
Buying @ 163.08 on 2020-12-18 00:00:00
Sell now @ 160.01 on 2020-12-23 00:00:00
Buying @ 161.72 on 2020-12-30 00:00:00
Sell now @ 171.88 on 2021-01-25 00:00:00
Buying @ 174.75 on 2021-02-02 00:00:00
Sell now @ 168.58 on 2021-02-03 00:00:00
Buying @ 172.0 on 2021-02-04 00:00:00
Sell now @ 169.93 on 2021-02-05 00:00:00
Buying @ 175.12 on 2021-02-08 00:00:00
Sell now @ 178.35 on 2021-02-19 00:00:00
[-2.72, -1.88, 6.28, -3.53, -1.2, 1.84]
     TOTAL GAINS/LOSSES: -1.2099999999999989%
****CHECKING STOCK MRAAY****
Buying @ 21.76 on 2020-12-03 00:00:00
Sell now @ 21.24 on 2020-12-07 00:00:00
Buying @ 21.59 on 2020-12-15 00:00:00
Sell

Buying @ 43.54 on 2021-01-06 00:00:00
Sell now @ 42.59 on 2021-01-21 00:00:00
Buying @ 42.04 on 2021-02-03 00:00:00
[-0.02, -2.18]
     TOTAL GAINS/LOSSES: -2.2%
****CHECKING STOCK DAL****
Buying @ 41.07 on 2020-12-02 00:00:00
Sell now @ 40.69 on 2020-12-14 00:00:00
Buying @ 40.21 on 2020-12-31 00:00:00
Sell now @ 38.73 on 2021-01-04 00:00:00
Buying @ 40.38 on 2021-01-06 00:00:00
Sell now @ 39.41 on 2021-01-11 00:00:00
Buying @ 40.3 on 2021-01-12 00:00:00
Sell now @ 38.9 on 2021-01-25 00:00:00
Buying @ 39.78 on 2021-02-03 00:00:00
[-0.93, -3.68, -2.4, -3.47]
     TOTAL GAINS/LOSSES: -10.48%
****CHECKING STOCK AAL****
Buying @ 14.86 on 2020-12-02 00:00:00
Sell now @ 17.01 on 2020-12-15 00:00:00
Buying @ 16.44 on 2021-01-14 00:00:00
Sell now @ 15.43 on 2021-01-25 00:00:00
Buying @ 16.56 on 2021-01-27 00:00:00
Sell now @ 16.99 on 2021-02-11 00:00:00
Buying @ 17.99 on 2021-02-17 00:00:00
[14.47, -6.14, 2.6]
     TOTAL GAINS/LOSSES: 10.930000000000001%
****CHECKING STOCK SAVE****
Buying @ 2

****CHECKING STOCK MLM****
Buying @ 280.67 on 2020-12-03 00:00:00
Sell now @ 266.29 on 2020-12-10 00:00:00
Buying @ 275.88 on 2020-12-18 00:00:00
Sell now @ 303.07 on 2021-01-15 00:00:00
Buying @ 300.4 on 2021-02-08 00:00:00
[-5.12, 9.86]
     TOTAL GAINS/LOSSES: 4.739999999999999%
****CHECKING STOCK NSC****
Buying @ 238.71 on 2020-12-02 00:00:00
Sell now @ 236.2 on 2020-12-03 00:00:00
Buying @ 239.22 on 2020-12-04 00:00:00
Sell now @ 235.55 on 2020-12-10 00:00:00
Buying @ 232.18 on 2020-12-23 00:00:00
Sell now @ 248.84 on 2021-01-19 00:00:00
Buying @ 245.26 on 2021-02-03 00:00:00
Sell now @ 254.1 on 2021-02-25 00:00:00
[-1.05, -1.53, 7.18, 3.6]
     TOTAL GAINS/LOSSES: 8.2%
****CHECKING STOCK UNP****
Buying @ 202.62 on 2020-12-02 00:00:00
Sell now @ 201.59 on 2020-12-10 00:00:00
Buying @ 202.66 on 2020-12-18 00:00:00
Sell now @ 200.25 on 2020-12-23 00:00:00
Buying @ 202.46 on 2020-12-24 00:00:00
Sell now @ 214.29 on 2021-01-19 00:00:00
Buying @ 203.42 on 2021-02-03 00:00:00
Sell now @

In [6]:
# Daily Buy and Sell lists

yesterday = []
sell_today = []
buy_today = []
#new_df = pd.DataFrame()

for i in ticker_dictionary:
    new_df = ticker_dictionary[i]
    new_df["Stock"] = str(i)
    last_rows = new_df.iloc[[-1], [9, 10, 11]]
    last_rows = last_rows.dropna(axis='rows', thresh=2)
    
    buy_stock = last_rows.dropna(subset = ["Buy_Signal_Price"])
    buy_stock = buy_stock[["Stock", "Buy_Signal_Price"]]
    buy_stock = buy_stock.rename_axis("", axis = 0)
    buy_stock.columns = [''] * len(buy_stock.columns)
    
    if not buy_stock.empty:
        buy_today.append(buy_stock)
        
    sell_stock = last_rows.dropna(subset = ["Sell_Signal_Price"])
#    sell_stock = sell_stock[["Sell_Signal_Price", "Stock"]]
    sell_stock = sell_stock[["Stock", "Sell_Signal_Price"]]
    sell_stock = sell_stock.rename_axis("", axis = 0)
    sell_stock.columns = [''] * len(sell_stock.columns)
    
    if not sell_stock.empty:
        sell_today.append(sell_stock)
    
    if not last_rows.empty:
        yesterday.append(last_rows)

print("Buy list: " + str(buy_today))
print("Sell list: " + str(sell_today))

IndexError: positional indexers are out-of-bounds

In [7]:
# Yagmail will only work if the sender_email uses a Gmail email. 

sender_email = os.environ.get("SENDER_EMAIL")
receiver_email = os.environ.get('RECEIVER_EMAIL')
sender_password = os.environ.get("SENDER_PASSWORD")

yag = yagmail.SMTP(user=sender_email, password=sender_password)

subject = "Daily Stock Trades"

contents = [
  '"I think you will find',
    "\n",
   'When your death takes its toll',
    "\n",
    'All the money you made',
    "\n",
    'Will never buy back your soul" - Bob Dylan',
    "\n",
    "Here is Today's Buy List: ", 
    "\n",
     str(buy_today),
    "\n",
    "Here is Today's Sell List: ",
    "\n", 
     str(sell_today),
    ]

yag.send(receiver_email, subject, contents)

{}